In [1]:
import pandas as pd
from fractions import Fraction

csv_path = "Data/mr_boston_flattened.csv"

## Read CSV file

In [2]:
#read csv
cocktail_df = pd.read_csv(csv_path, encoding="utf-8")

## Define function ## 
to convert string values to number and clean measurement

In [3]:
#function for converting fractions and data cleanup
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        frac_str = frac_str.lower()
        if "or" in frac_str:
            nums = frac_str.split(' or ')
            return nums[0]
        elif "  " in frac_str:
            frac_str = frac_str.replace("  ", " ")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value
        elif "tsp" in frac_str:
            frac_str = frac_str.strip("tsp")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 0.166667
        elif "750-ml" in frac_str:
            frac_str = frac_str.strip("750-ml")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 25.3
        elif "750ml" in frac_str:
            frac_str = frac_str.strip("750ml")
            print(frac_str)
            return 25.3
        elif "bottles" in frac_str:
            frac_str = frac_str.strip("bottles")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 25.3
        elif "for glass" in frac_str:
            nums = frac_str.split('for glass')
            return 1
        elif "/" in frac_str:
            num, denom = frac_str.split('/')
            try:
                leading, num = num.split(' ')
                whole = float(leading)
            except ValueError:
                whole = 0
            frac = float(num) / float(denom)
            return whole - frac if whole < 0 else whole + frac    
        else:
            print("from else")
            print(frac_str)
            return frac_str

## Combine all Ingredient and Measure Columns

In [4]:
#create Dataframes for each ingredient subset
ing1_df = cocktail_df[["name", "measurement-1", "ingredient-1"]]
ing1 = ing1_df.rename(columns= {"measurement-1": "measurement", "ingredient-1":"ingredient"})
ing2_df = cocktail_df[["name", "measurement-2", "ingredient-2"]]
ing2 = ing2_df.rename(columns= {"measurement-2": "measurement", "ingredient-2":"ingredient"})
ing3_df = cocktail_df[["name", "measurement-3", "ingredient-3"]]
ing3 = ing3_df.rename(columns= {"measurement-3": "measurement", "ingredient-3":"ingredient"})
ing4_df = cocktail_df[["name", "measurement-4", "ingredient-4"]]
ing4 = ing4_df.rename(columns= {"measurement-4": "measurement", "ingredient-4":"ingredient"})
ing5_df = cocktail_df[["name", "measurement-5", "ingredient-5"]]
ing5 = ing5_df.rename(columns= {"measurement-5": "measurement", "ingredient-5":"ingredient"})
ing6_df = cocktail_df[["name", "measurement-6", "ingredient-6"]]
ing6 = ing6_df.rename(columns= {"measurement-6": "measurement", "ingredient-6":"ingredient"})

In [5]:
#combine data frames and sort
frames = [ing1, ing2, ing3, ing4, ing5, ing6]

working_df = pd.concat(frames)
working_df

,name,measurement,ingredient
0,Gauguin,2 oz,Light Rum
1,Fort Lauderdale,1 1/2 oz,Light Rum
2,Apple Pie,3 oz,Apple schnapps
3,Cuban Cocktail No. 1,1/2 oz,Juice of a Lime
4,Cool Carlos,1 1/2 oz,Dark rum
...,...,...,...
985,Wallis Blue Cocktail,NaN,NaN
986,Minnehaha Cocktail,NaN,NaN
987,Wallick Cocktail,NaN,NaN
988,Waikiki Beachcomber,NaN,NaN


In [6]:
working_df = working_df.dropna(subset =['ingredient'])
reset_df = working_df.reset_index(inplace= False)
reset2_df = reset_df.drop('index', 1)
reset2_df['measurement'] = reset2_df['measurement'].fillna(1)
reset2_df 


,name,measurement,ingredient
0,Gauguin,2 oz,Light Rum
1,Fort Lauderdale,1 1/2 oz,Light Rum
2,Apple Pie,3 oz,Apple schnapps
3,Cuban Cocktail No. 1,1/2 oz,Juice of a Lime
4,Cool Carlos,1 1/2 oz,Dark rum
...,...,...,...
3929,Bloody Maria,1,Lemon wheel
3930,Amante Picante,2 oz,"Jalapeno hot pepper sauce, cucumber slice"
3931,Betsy Ross Cocktail,1 1/4 oz,Old Mr. Boston Dry Gin
3932,The Winkle,4 oz,"Raspberries, lemon twist"


In [7]:
reset3_df = reset2_df
reset3_df['liquid'] = reset3_df['measurement'].str.contains(' oz')
reset3_df['liquid'] = reset3_df['liquid'].fillna('False')
reset3_df['unit'] = ""

In [8]:
for row, index in reset3_df.iterrows():
        if reset3_df['liquid'][row] == True:
            reset3_df['unit'][row] = ("Fl Oz")
        else:
            reset3_df['unit'][row] = ("garnish")
            

In [9]:
reset3_df.to_csv("Data/ingredients.csv", index=False, header=True, encoding="UTF-8")
reset3_df

,name,measurement,ingredient,liquid,unit
0,Gauguin,2 oz,Light Rum,True,Fl Oz
1,Fort Lauderdale,1 1/2 oz,Light Rum,True,Fl Oz
2,Apple Pie,3 oz,Apple schnapps,True,Fl Oz
3,Cuban Cocktail No. 1,1/2 oz,Juice of a Lime,True,Fl Oz
4,Cool Carlos,1 1/2 oz,Dark rum,True,Fl Oz
...,...,...,...,...,...
3929,Bloody Maria,1,Lemon wheel,False,garnish
3930,Amante Picante,2 oz,"Jalapeno hot pepper sauce, cucumber slice",True,Fl Oz
3931,Betsy Ross Cocktail,1 1/4 oz,Old Mr. Boston Dry Gin,True,Fl Oz
3932,The Winkle,4 oz,"Raspberries, lemon twist",True,Fl Oz


In [10]:
#strip oz
reset3_df['measurement']= reset3_df['measurement'].str.strip(" oz")
reset3_df

,name,measurement,ingredient,liquid,unit
0,Gauguin,2,Light Rum,True,Fl Oz
1,Fort Lauderdale,1 1/2,Light Rum,True,Fl Oz
2,Apple Pie,3,Apple schnapps,True,Fl Oz
3,Cuban Cocktail No. 1,1/2,Juice of a Lime,True,Fl Oz
4,Cool Carlos,1 1/2,Dark rum,True,Fl Oz
...,...,...,...,...,...
3929,Bloody Maria,NaN,Lemon wheel,False,garnish
3930,Amante Picante,2,"Jalapeno hot pepper sauce, cucumber slice",True,Fl Oz
3931,Betsy Ross Cocktail,1 1/4,Old Mr. Boston Dry Gin,True,Fl Oz
3932,The Winkle,4,"Raspberries, lemon twist",True,Fl Oz


## Run function and reindex

In [11]:
reset3_df.to_csv("Data/ingredients.csv", index=False, header=True, encoding="UTF-8")
reset3_df

,name,measurement,ingredient,liquid,unit
0,Gauguin,2,Light Rum,True,Fl Oz
1,Fort Lauderdale,1 1/2,Light Rum,True,Fl Oz
2,Apple Pie,3,Apple schnapps,True,Fl Oz
3,Cuban Cocktail No. 1,1/2,Juice of a Lime,True,Fl Oz
4,Cool Carlos,1 1/2,Dark rum,True,Fl Oz
...,...,...,...,...,...
3929,Bloody Maria,NaN,Lemon wheel,False,garnish
3930,Amante Picante,2,"Jalapeno hot pepper sauce, cucumber slice",True,Fl Oz
3931,Betsy Ross Cocktail,1 1/4,Old Mr. Boston Dry Gin,True,Fl Oz
3932,The Winkle,4,"Raspberries, lemon twist",True,Fl Oz


In [12]:
#apply function to convert fractions
reset3_df['measurement'] = reset3_df['measurement'].apply(convert_to_float)
reset3_df['measurement'] = reset3_df['measurement'].fillna(1)

1/2 
1 
2 
2 
from else
3 slices
from else
6 fresh
from else
splash
from else
3 slices
1 3/4
from else
1 splash
from else
3 slices
from else

from else
2 dashes
from else
1c
1 
2 - 3 drops
from else
2 - 3 drops
from else
1 dash


In [13]:
ingred_df = reset3_df
final_ingred_df = ingred_df.rename(columns={'name':'cocktail'})
final_ingred_df

,cocktail,measurement,ingredient,liquid,unit
0,Gauguin,2,Light Rum,True,Fl Oz
1,Fort Lauderdale,1.5,Light Rum,True,Fl Oz
2,Apple Pie,3,Apple schnapps,True,Fl Oz
3,Cuban Cocktail No. 1,0.5,Juice of a Lime,True,Fl Oz
4,Cool Carlos,1.5,Dark rum,True,Fl Oz
...,...,...,...,...,...
3929,Bloody Maria,1,Lemon wheel,False,garnish
3930,Amante Picante,2,"Jalapeno hot pepper sauce, cucumber slice",True,Fl Oz
3931,Betsy Ross Cocktail,1.25,Old Mr. Boston Dry Gin,True,Fl Oz
3932,The Winkle,4,"Raspberries, lemon twist",True,Fl Oz


In [14]:
final_list_df = final_ingred_df.drop('liquid', 1)

array([' Light Rum', ' Apple schnapps', ' Juice of a Lime', ' Dark rum',
       ' Bourbon whiskey', ' Amaretto', ' Scotch Whiskey', ' Bacardi Rum',
       ' Brandy', ' Gin', ' Sloe gin', ' Sweet Vermouth',
       ' Tanqueray gin', ' Lemon Juice', ' Straight rye whiskey',
       ' Grenadine', ' Green Chartreuse', ' Irish Whiskey',
       ' Dry Vermouth', ' Juice of a Lemon', ' Orange', ' Absinthe',
       ' Rye or bourbon whiskey', ' Light Vermouth', ' Anisette',
       ' Kummel', ' Bourbon or Rye Whiskey', ' Tennessee whiskey',
       ' Vodka', ' Grapefruit Juice', ' Fresh rosemary sprig',
       ' Powdered Sugar', ' Wide spiral of lemon zest',
       ' lemon-flavored vodka', ' Lime Juice', ' Blended Scotch Whiskey',
       ' Apricot Flavored Brandy', ' Bitters', ' Juice of Orange',
       ' Orange Bitters', 'Blended Scotch Whiskey', ' White whiskey',
       ' Dry gin', ' Forbidden Fruit', ' Applejack', ' Port',
       ' Apple Brandy', ' Maraschino', ' Coffee-flavored brandy',
       '

## Postgres for Ingredient table

In [16]:
final_list_df.to_csv("Data/ingredients.csv", index=False, header=True, encoding="UTF-8")

In [17]:
from sqlalchemy import create_engine
from config import postgres_pwd, postgres_user

db_name = "cocktail_creator"
table_name = "measure"

connection_string = f"{postgres_user}:{postgres_pwd}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()

final_list_df.to_sql(table_name, con=engine, if_exists="append", index=False)

conn.close()

ImportError: cannot import name 'postgres_pwd' from 'config' (/Users/timsamson/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/config/__init__.py)